In [1]:
%matplotlib inline

import matplotlib
matplotlib.style.use('ggplot')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math

//anaconda/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


In [2]:
from bs4 import BeautifulSoup
import requests

url = "https://marketdata.set.or.th/mkt/sectorquotation.do?sector=SET100&language=en&country=US"
page = requests.get(url)
data = page.content
soup = BeautifulSoup(data)
SET100 = {}

for i,link in enumerate(soup.find_all('td', style="text-align: left;")):
    for sublink in link.find_all('a'):
        html = sublink.get('href')
        if html[:14] == "stockquotation":
            name = html.split("stockquotation.do?symbol=")[1].split("&sso")[0]
            SET100[name] = {}
            SET100[name]["PE"] = {}
            SET100[name]["EPS"] = {}
            SET100[name]["ProfitMargin"] = {}
        else:
            pass

//anaconda/lib/python2.7/site-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 174 of the file //anaconda/lib/python2.7/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "lxml")

  markup_type=markup_type))


In [3]:
def ProCut(s):
    check = False
    ans = []
    s1 = ""
    for i in s:
        if i == ">":
            check = True
            s1 = ""
        elif i == "&" and s1 != "" and s1 != "&nbsp;":
            ans.append(s1)
            s1 = ""
            check = False
        elif check:
            s1 += i
        else:
            continue
    return ans

In [4]:
def getPE(stockname):
    url = "https://www.set.or.th/set/companyhighlight.do?symbol="+stockname+"&ssoPageId=5&language=en&country=US"
    page = requests.get(url)
    data = page.content
    soup = BeautifulSoup(data)
    test = data.find('''<td style="text-align:left;" height="15">P/E</td>''')
    PE = data[test+50:test+250]
    return ProCut(PE)
    
def getEPS(stockname):
    url = "https://www.set.or.th/set/companyhighlight.do?symbol="+stockname+"&ssoPageId=5&language=en&country=US"
    page = requests.get(url)
    data = page.content
    soup = BeautifulSoup(data)
    test = data.find('''<td style="text-align:left;" height="15">EPS (Baht)</td>''')
    EPS = data[test+50:test+200]
    return ProCut(EPS)

def getNPM(stockname):
    url = "https://www.set.or.th/set/companyhighlight.do?symbol="+stockname+"&ssoPageId=5&language=en&country=US"
    page = requests.get(url)
    data = page.content
    soup = BeautifulSoup(data)
    test = data.find('''<td style="text-align:left;" height="15">Net Profit Margin(%)</td>''')
    NPM = data[test+50:test+200]
    return ProCut(NPM)


In [5]:
for a in SET100:
    print a
    FullPE = getPE(a)
    FullEPS = getEPS(a)
    FullNPM = getNPM(a)
    yearPE = ["2017","2016","2015","2014","2013","2012"]
    yearEPS = ["2016","2015","2014","2013","2012"]
    for i in range(len(FullPE)):
        SET100[a]["PE"][yearPE[i]] = FullPE[(-i-1)]
    for i in range(len(FullEPS)):
        SET100[a]["EPS"][yearEPS[i]] = FullEPS[(-i-1)]
    for i in range(len(FullNPM)):
        SET100[a]["ProfitMargin"][yearEPS[i]] = FullNPM[(-i-1)]

BA
CPN
BH
CPF
CKP
EPG
THAI
TRUE
VNG
RS
AOT
CHG
HANA
LPN
GPSC
GLOW
CENTEL
IRPC
GUNKUL
IFEC
THCOM
BEM
BEC
ROBINS
SCN
PTG
TTW
SCC
SCB
BLAND
TTCL
TTA
PTT
CPALL
VGI
THANI
SIRI
PTTEP
HMPRO
UNIQ
BLA
KCE
PSH
CBG
BTS
IVL
SPALI
SPCG
KBANK
KKP
SPRC
BCH
TASCO
STPI
GLOBAL
ICHI
TMB
SAMART
BCP
WHA
COM7
TVO
INTUCH
KAMART
CK
BBL
PTTGC
EGCO
MINT
TCAP
MAJOR
VIBHA
ITD
TPIPL
MTLS
BDMS
STEC
ADVANC
S
TOP
AAV
DTAC
LHBANK
SGP
TU
LH
TISCO
BEAUTY
DELTA
AP
BANPU
AMATA
KTB
KTC
TKN
PLANB
SAWAD
BIG
QH
SUPER


In [9]:
StockComponent = {}
check = []
count = 0

for i in SET100:
    try:
        print 'Stock:',i
        StockDf = pd.DataFrame(SET100[i])
        print StockDf
        StockComponent[i] = {}
        CurrentPE = float(StockDf.loc['2017','PE'])
        StockComponent[i]['MinPE'] = CurrentPE - (CurrentPE * 0.15)
        StockComponent[i]['MaxPE'] = CurrentPE + (CurrentPE * 0.15)
    except:
        check.append(i)
    print '==================================='


Stock: BA
       EPS     PE ProfitMargin
2014  0.21    N/A         1.56
2015  0.86  24.81         7.07
2016  0.84  22.23         6.44
2017   NaN  23.63          NaN
Stock: CPN
       EPS     PE ProfitMargin
2013  1.42  32.56        28.10
2014  1.63  28.22        29.50
2015  1.76  27.08        29.60
2016  2.06  28.50        30.70
2017   NaN  27.55          NaN
Stock: BH
       EPS     PE ProfitMargin
2013  3.46  27.87        17.20
2014  3.75  38.58        17.16
2015  4.72  45.91        19.15
2016  4.98  37.01        20.00
2017   NaN  36.34          NaN
Stock: CPF
       EPS     PE ProfitMargin
2013  0.96  44.66         1.75
2014  1.43  18.33         2.41
2015  1.50  13.73         2.52
2016  1.99  15.74         3.08
2017   NaN  14.61          NaN
Stock: CKP
       EPS      PE ProfitMargin
2013  0.22   62.29         3.87
2014  0.43   70.62         6.68
2015  0.06   31.28         5.98
2016  0.01   66.88         0.86
2017   NaN  431.05          NaN
Stock: EPG
       EPS     PE ProfitMargin


In [74]:
for i in SET100:
    try:
        StockDf = pd.DataFrame(SET100[i])
        SumPE = 0
        SumYear = 0
        PredictedEPS = 0
        ChangeEPS = []
        MinPE = "Min:",StockComponent[i]["MinPE"]
        MaxPE = "Max:",StockComponent[i]["MaxPE"]
        print "------------------"
        for j in xrange(len(StockDf.loc[:, 'EPS'])):
            priorEPS = StockDf.loc[:, 'EPS'][j]
            laterEPS = StockDf.loc[:, 'EPS'][j+1]
            if math.isnan(float(priorEPS)):
                pass
            elif math.isnan(float(laterEPS)):
                avgChange = sum(ChangeEPS) / float(len(ChangeEPS))
                currentEPS = float(priorEPS)
                PredictedEPS = currentEPS + ((avgChange * currentEPS) / 100.)
                StockComponent[i]["PredictedEPS"] = PredictedEPS
                print currentEPS, PredictedEPS
                print 'i:',i, avgChange
            else:
                YYChange = ((float(laterEPS) - float(priorEPS)) * 100.) / float(priorEPS)
                ChangeEPS.append(YYChange)
                print 'change:',YYChange
        for j in StockDf.loc[:,'PE']:
            if math.isnan(float(j)):
                pass
            else:
                if float(j) > float(MinPE[1]) and float(j) < float(MaxPE[1]):
                    SumPE += float(j)
                    SumYear += 1
#         print i
#         print "Average PE:", "{0:.2f}".format(SumPE / SumYear)
    except:
        pass


------------------
change: 309.523809524
change: -2.32558139535
0.84 2.13023255814
i: BA 153.599114064
------------------
change: 14.7887323944
change: 7.9754601227
change: 17.0454545455
2.06 2.3333595765
i: CPN 13.2698823542
------------------
change: 8.38150289017
change: 25.8666666667
change: 5.50847457627
4.98 5.63996029261
i: BH 13.252214711
------------------
change: 48.9583333333
change: 4.8951048951
change: 32.6666666667
1.99 2.5639166958
i: CPF 28.840034965
------------------
change: 95.4545454545
change: -86.0465116279
change: -83.3333333333
0.01 0.00753582334978
i: CKP -24.6417665022
------------------
change: 85.1851851852
change: -20.0
0.4 0.53037037037
i: EPG 32.5925925926
------------------
change: -420.0
change: 89.5833333333
change: 12.0879120879
1.02 -0.0623177655678
i: VNG -106.10958486
------------------
change: -25.2622377622
change: 53.3333333333
change: 4.50038138825
13.7 15.1874307811
i: AOT 10.8571589864
------------------
change: 7.31707317073
change: -88.6363

In [76]:
pd.DataFrame(StockComponent)

,AAV,ADVANC,AMATA,AOT,AP,BA,BANPU,BBL,BCH,BCP,...,TTA,TTCL,TTW,TU,TVO,UNIQ,VGI,VIBHA,VNG,WHA
MaxPE,18.641500,19.619000,16.180500,31.786000,9.3725,27.174500,67.321000,12.76500,49.887000,10.948000,...,NaN,27.197500,19.458000,21.930500,12.489000,26.312000,47.667500,66.470000,15.628500,17.273000
MinPE,13.778500,14.501000,11.959500,23.494000,6.9275,20.085500,49.759000,9.43500,36.873000,8.092000,...,NaN,20.102500,14.382000,16.209500,9.231000,19.448000,35.232500,49.130000,11.551500,12.767000
PredictedEPS,0.969091,9.838888,1.118253,15.187431,0.9236,2.130233,-0.075825,16.02324,0.334161,8.206702,...,4.221958,0.598286,0.614328,0.895345,4.920369,0.860381,0.075944,0.032336,-0.062318,0.136558
